## Note that 'librosa' ONLY runs on Python 3.9.x, and thus more recent versions of Python kernels would not work.

In [1]:
!python3.9 -m venv myenv
!source myenv/bin/activate
!pip install --upgrade pip
!pip install notebook ipykernel
!python -m ipykernel install --user --name=myenv --display-name "Python (myenv)"

/bin/bash: line 1: python3.9: command not found
/bin/bash: line 1: myenv/bin/activate: No such file or directory
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.3 MB/s eta 0:00:00 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Installed kernelspec myenv in /home/hice1/hyang491/.local/share/jupyter/kernels/myenv


In [2]:
import os
import re
import librosa
import noisereduce as nr
import numpy as np
import sys
from scipy.io import wavfile
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.utils import Bunch

In [3]:
# get location of audio files
root_directory = os.getcwd()

In [4]:

audio_directory = os.path.join(root_directory, 'ravdess')
nr_directory = os.path.join(root_directory, 'ravdess_NR')
if os.path.exists(nr_directory):
    print("'ravdess_NR' folder already exists, will not run noise reduction; to run it, please delete the file and try again.")
else:
    # Will raise error if folder exists; repetitive, but included as fail-safe
    os.makedirs(nr_directory, exist_ok=False)
    for root, _, files in os.walk(audio_directory):
        relative_path = os.path.relpath(root, audio_directory)
        save_directory = os.path.join(nr_directory, relative_path)
        os.makedirs(save_directory, exist_ok=True)
        print(f"Created directory: {save_directory}")
        
        for file_name in files:
            file_path = os.path.join(root, file_name)
            if file_name.endswith('.wav'):
                rate, data = wavfile.read(file_path)
                # 1) data is mono-channel
                if data.ndim == 1:
                    reduced_noise = nr.reduce_noise(y=data, sr=rate)
                # 2) data is multi-channel
                else:
                    reduced_noise = []
                    for i in range(data.shape[1]):
                        reduced_channel = nr.reduce_noise(y=data[:, i], sr=rate)
                        reduced_noise.append(reduced_channel)
                    reduced_noise = np.stack(reduced_noise, axis=1)
                reduced_noise = reduced_noise.astype(data.dtype)
                reduce_name = f'{os.path.splitext(file_name)[0]}_NR.wav'
                wavfile.write(os.path.join(save_directory, reduce_name), rate, reduced_noise)
        print(f"Completed noise reduction for files in folder {root}")

Created directory: /storage/ice1/7/4/hyang491/PCA_Audio/ravdess_NR/.
Completed noise reduction for files in folder /storage/ice1/7/4/hyang491/PCA_Audio/ravdess
Created directory: /storage/ice1/7/4/hyang491/PCA_Audio/ravdess_NR/Actor_21


/scratch/1001702/ipykernel_497290/1751505744.py:17: WavFileWarning: Chunk (non-data) not understood, skipping it.
  rate, data = wavfile.read(file_path)


Completed noise reduction for files in folder /storage/ice1/7/4/hyang491/PCA_Audio/ravdess/Actor_21
Created directory: /storage/ice1/7/4/hyang491/PCA_Audio/ravdess_NR/Actor_16
Completed noise reduction for files in folder /storage/ice1/7/4/hyang491/PCA_Audio/ravdess/Actor_16
Created directory: /storage/ice1/7/4/hyang491/PCA_Audio/ravdess_NR/Actor_07
Completed noise reduction for files in folder /storage/ice1/7/4/hyang491/PCA_Audio/ravdess/Actor_07
Created directory: /storage/ice1/7/4/hyang491/PCA_Audio/ravdess_NR/Actor_14
Completed noise reduction for files in folder /storage/ice1/7/4/hyang491/PCA_Audio/ravdess/Actor_14
Created directory: /storage/ice1/7/4/hyang491/PCA_Audio/ravdess_NR/Actor_23
Completed noise reduction for files in folder /storage/ice1/7/4/hyang491/PCA_Audio/ravdess/Actor_23
Created directory: /storage/ice1/7/4/hyang491/PCA_Audio/ravdess_NR/Actor_13
Completed noise reduction for files in folder /storage/ice1/7/4/hyang491/PCA_Audio/ravdess/Actor_13
Created directory: /

In [5]:
!{sys.executable} -m pip install librosa

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [6]:
def pad_signal(signal, target_length=512):
    if len(signal) < target_length:
        padding = target_length - len(signal)
        signal = np.pad(signal, (0, padding), 'constant')
    return signal

In [7]:
def extract_features(file_path, sample_rate=48000, n_fft=512):
    signal, sr = librosa.load(file_path, sr=sample_rate)
    signal = librosa.effects.trim(signal)[0]
    signal = pad_signal(signal, target_length=n_fft)

    mfccs = np.mean(librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13, n_fft=n_fft).T, axis=0)
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=signal, sr=sr, n_fft=n_fft), axis=1)
    mel_spectrogram = np.mean(librosa.feature.melspectrogram(y=signal, sr=sr, n_mels=40, n_fft=n_fft), axis=1)
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=signal, sr=sr, n_fft=n_fft))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=signal, sr=sr, n_fft=n_fft))
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=signal, sr=sr, n_fft=n_fft), axis=1)
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=signal, sr=sr, n_fft=n_fft))

    tonnetz = np.mean(librosa.feature.tonnetz(y=signal, sr=sr), axis=1)
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=signal))
    chroma_cqt = np.mean(librosa.feature.chroma_cqt(y=signal, sr=sr), axis=1)
    chroma_cens = np.mean(librosa.feature.chroma_cens(y=signal, sr=sr), axis=1)
    spectral_flatness = np.mean(librosa.feature.spectral_flatness(y=signal))
    rms = np.mean(librosa.feature.rms(y=signal))

    features = np.hstack([
        mfccs, chroma_stft, mel_spectrogram, spectral_centroid, spectral_bandwidth, 
        tonnetz, zcr, chroma_cqt, chroma_cens, spectral_contrast, 
        spectral_flatness, spectral_rolloff, rms
    ])

    return features

In [ ]:
# Function that creates sklearn dataset
# input: audio_directory = path of audio data
# output: original_dataset = dataset of features, labels = labels
def create_dataset(audio_directory):
    file_paths = []
    labels = []

    # pattern to match folder and file names(avoid file errors)
    folderPattern = re.compile(r"Actor_\d{2}")
    filePattern = re.compile(r"\d{2}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}_NR\.wav")

    for root, _, files in os.walk(audio_directory):
        if folderPattern.match(os.path.basename(root)):
            for file_name in files:
                if filePattern.match(file_name):
                    emotion = int(file_name.split("-")[2])
                    labels.append(emotion)
                    file_paths.append(os.path.join(root, file_name))

    # get features for all files
    all_features = []
    for file in file_paths:
        all_features.append(extract_features(file))

    # convert list of features and labels to np array and create dataset
    all_features = np.array(all_features)
    labels = np.array(labels)

    return all_features, labels

In [ ]:
# Function that runs PCA on sklearn dataset
# input: orig_dataset = original dataset of features, labels = labels, target_variance = default 0.95
# output: pca_features_reduced = reduced dataset of features, labels = labels
def pca_dataset(nr_features, nr_labels, target_variance=0.95):
    # # standardize dataset and apply PCA
    scaler = StandardScaler()
    all_features_scaled = scaler.fit_transform(nr_features)
    pca = PCA(target_variance)
    pca_features_reduced = pca.fit_transform(all_features_scaled)

    print(f"Original number of features: {nr_features.shape[1]}")
    print(f"Reduced number of features with {target_variance * 100}% variance retained: {pca_features_reduced.shape[1]}")

    return pca_features_reduced, nr_labels

In [10]:
import pickle
# check if there is pickle file for nonreduced dataset; if exists, import data
nonreduced_db_pickle_path = os.path.join(root_directory, 'nonreduced')
print(nonreduced_db_pickle_path)
if os.path.exists(nonreduced_db_pickle_path):
    print("'nonreduced.pkl' file already exists, will load file and not create new dataset; to run it, please delete the file and try again.")
    nonreduced_db_file = open('nonreduced', 'rb')    
    nonreduced_db = pickle.load(nonreduced_db_file)
    nr_features, nr_labels = nonreduced_db
    nonreduced_db_file.close()
# else create dataset and resulting pickle file
else:
    nonreduced_db = create_dataset(nr_directory)
    nr_features, nr_labels = nonreduced_db
    # save dataset as pickle file
    nonreduced_db_file = open('nonreduced', 'ab')
    # source, destination
    pickle.dump(nonreduced_db, nonreduced_db_file)                    
    nonreduced_db_file.close()

# print dataset
print(nr_features)
print(nr_labels)
print(np.shape(nr_features))
print(np.shape(nr_labels))

/storage/ice1/7/4/hyang491/PCA_Audio/nonreduced


/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=680
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=832
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of le

/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=792
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=552
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=604
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=872
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=808
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=852
  return f(*args, **kwargs

/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=944
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=472
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=476
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=992
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=496
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=984
  return f(*args, **kwargs

/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=532
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=444
  return f(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=1012
  return f(*args, **kwargs)


[[-7.49839539e+02  6.20220375e+01 -1.19297285e+01 ...  3.48926522e-02
   4.63925439e+03  1.20826857e-02]
 [-7.24241333e+02  7.22653656e+01 -2.62639294e+01 ...  3.88392955e-02
   3.98549641e+03  1.33437859e-02]
 [-8.68214905e+02  4.86408463e+01 -7.70047998e+00 ...  7.03973696e-02
   6.12351896e+03  3.77656193e-03]
 ...
 [-1.03060815e+03  3.42490959e+01 -1.55566275e+00 ...  4.64618415e-01
   2.51250000e+03  8.05703632e-04]
 [-7.77870605e+02  6.24671097e+01 -6.52451754e+00 ...  4.18793783e-02
   4.58490566e+03  6.81798020e-03]
 [-9.85018250e+02  3.38118591e+01  4.86478861e-03 ...  4.98436421e-01
   2.49066695e+03  1.40213128e-03]]
[7 4 2 ... 2 5 1]
(1440, 108)
(1440,)


In [11]:
# check if there is pickle file for reduced dataset; if exists, import data
reduced_db_pickle_path = os.path.join(root_directory, 'reduced')
if os.path.exists(reduced_db_pickle_path):
    print("'reduced.pkl' file already exists, will load file and not create new dataset; to run it, please delete the file and try again.")
    reduced_db_file = open('reduced', 'rb')    
    reduced_db = pickle.load(reduced_db_file)
    r_features, r_labels = reduced_db
    reduced_db_file.close()
# else run PCA on existing dataset and create pickle file
else:
    reduced_db = pca_dataset(nr_features, nr_labels)
    r_features, r_labels = reduced_db
    # save dataset as pickle file
    reduced_db_file = open('reduce', 'ab')
    # source, destination
    pickle.dump(reduced_db, reduced_db_file)                    
    reduced_db_file.close()

# print dataset
print(r_features)
print(r_labels)
print(np.shape(r_features))
print(np.shape(r_labels))

Original number of features: 108
Reduced number of features with 95.0% variance retained: 37
[[ 9.59421374e-01 -2.13160858e+00  7.25211966e-01 ... -4.43656085e-01
   4.28159844e-01 -7.48643626e-01]
 [ 9.38911866e-01 -1.93406652e+00 -3.22610990e+00 ... -7.69578463e-01
   7.71518297e-02 -2.28676000e-01]
 [-6.91222952e-01 -2.48232334e+00 -2.01013177e-01 ... -1.59066861e-01
   4.83167208e-01 -7.23641057e-01]
 ...
 [-6.30042336e+00  4.19348328e+00 -1.45964726e+00 ... -8.47781936e-02
  -3.91546246e-01 -8.90814901e-02]
 [-6.57980910e-02 -2.93522051e+00 -2.86158682e+00 ... -1.02629296e-01
  -2.66436494e-01  5.60885519e-03]
 [-6.87323239e+00  5.30681823e+00 -1.03469158e+00 ... -1.27583030e-01
  -2.32364580e-01  3.43872521e-01]]
[7 4 2 ... 2 5 1]
(1440, 37)
(1440,)
